### Data import and libraries

In [3]:
import pandas as pd

segmentation_st = pd.read_sas("data_for_segmentation_st.sas7bdat")
segmentation_mw = pd.read_sas("data_for_segmentation_wi.sas7bdat")

In [7]:
import numpy as np
from chefboost.training import Training as Training
from chefboost import Chefboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [12]:
segmentation_st["performance"].value_counts()

performance
G    83990
B     2374
Name: count, dtype: int64

In [16]:
segmentation_mw["performance"].value_counts()

performance
G    430775
B     26625
Name: count, dtype: int64

### Bytes to strings

In [4]:
columns_to_convert = ["cbsmessageid","C_600570","C_660570","C_500300","C_500600","C_501200","C_502400","performance","Exclusion"]
def convert_bytes_to_str(x):
    return x.decode('utf-8') if isinstance(x, bytes) else x
segmentation_st[columns_to_convert] = segmentation_st[columns_to_convert].map(convert_bytes_to_str)
segmentation_mw[columns_to_convert] = segmentation_mw[columns_to_convert].map(convert_bytes_to_str)

In [5]:
segmentation_st = segmentation_st.drop(["cbsmessageid","Exclusion"],axis=1)
segmentation_mw = segmentation_mw.drop(["cbsmessageid","Exclusion"],axis=1)

### Train test split

#### ST

In [10]:
features = list(segmentation_st.drop("performance",axis=1).columns)
target = "performance"

In [57]:
X_train, X_test = train_test_split(segmentation_st, test_size=0.3, random_state=42)

#### MW

In [8]:
X_train_mw, X_test_mw = train_test_split(segmentation_mw, test_size=0.3, random_state=42)

### Chaid

In [58]:
#st
config = {"algorithm": "CHAID"}
model = cb.fit(X_train,config,target_label=target)

25-02-14 14:12:32 - CHAID tree is going to be built...
25-02-14 14:19:06 - -------------------------
25-02-14 14:19:06 - finished in 394.36629605293274 seconds
25-02-14 14:19:13 - -------------------------
25-02-14 14:19:13 - Evaluate train set
25-02-14 14:19:13 - -------------------------
25-02-14 14:19:13 - Accuracy: 97.24583981208853% on 60454 instances
25-02-14 14:19:13 - Labels: ['G' 'B']
25-02-14 14:19:13 - Confusion matrix: [[58781, 1665], [0, 8]]
25-02-14 14:19:13 - Precision: 97.2455%, Recall: 100.0%, F1: 98.6035%


In [24]:
#mw
config = {"algorithm": "CHAID",'max_depth':3,'enableParallelism': True}
model_mw = cb.fit(X_train_mw,config,target_label=target)

25-02-14 16:40:07 - [INFO]: 7 CPU cores will be allocated in parallel running
25-02-14 16:40:07 - CHAID tree is going to be built...
25-02-14 16:50:46 - -------------------------
25-02-14 16:50:46 - finished in 639.1365330219269 seconds
25-02-14 16:51:29 - -------------------------
25-02-14 16:51:29 - Evaluate train set
25-02-14 16:51:29 - -------------------------
25-02-14 16:51:29 - Accuracy: 94.21606596289587% on 320180 instances
25-02-14 16:51:29 - Labels: ['G' 'B']
25-02-14 16:51:29 - Confusion matrix: [[301576, 18515], [4, 85]]
25-02-14 16:51:29 - Precision: 94.2157%, Recall: 99.9987%, F1: 97.0211%


#### ST prediction

In [60]:
# Predikce na testovací sadě
y_pred = []
for index, instance in X_test.iterrows():
    prediction = cb.predict(model, instance)
    y_pred.append(prediction)

# Vyhodnocení modelu
y_test = X_test[target]
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

c:\Users\cza187adebec\AppData\Local\anaconda3\Lib\site-packages\chefboost\Chefboost.py:347: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.isna(param[column_index]):
c:\Users\cza187adebec\AppData\Local\anaconda3\Lib\site-packages\chefboost\Chefboost.py:348: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  param[column_index] = missing_value
c:\Users\cza187adebec\OneDrive - CRIF SpA\Dokumenty\Python\Chaid\outputs\rules\rules.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To

Accuracy: 0.9727518332690082
              precision    recall  f1-score   support

           B       0.00      0.00      0.00       701
           G       0.97      1.00      0.99     25209

    accuracy                           0.97     25910
   macro avg       0.49      0.50      0.49     25910
weighted avg       0.95      0.97      0.96     25910



#### MW prediction

In [25]:
# Predikce na testovací sadě
y_pred_mw = []
for index, instance in X_test_mw.iterrows():
    prediction = cb.predict(model_mw, instance)
    y_pred_mw.append(prediction)

# Vyhodnocení modelu
y_test_mw = X_test_mw[target]
accuracy = accuracy_score(y_test_mw, y_pred_mw)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test_mw, y_pred_mw))

c:\Users\cza187adebec\AppData\Local\anaconda3\Lib\site-packages\chefboost\Chefboost.py:347: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.isna(param[column_index]):
c:\Users\cza187adebec\AppData\Local\anaconda3\Lib\site-packages\chefboost\Chefboost.py:348: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  param[column_index] = missing_value
c:\Users\cza187adebec\OneDrive - CRIF SpA\Dokumenty\Python\Chaid\outputs\rules\rules.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To

Accuracy: 0.9415464218044017
              precision    recall  f1-score   support

           B       0.58      0.00      0.00      8025
           G       0.94      1.00      0.97    129195

    accuracy                           0.94    137220
   macro avg       0.76      0.50      0.49    137220
weighted avg       0.92      0.94      0.91    137220



In [ ]:
# Uložení modelu
cb.save_model(model_mw, "model_mw_less_depth.pkl")
